# This notebook to demonstrate create a new user on Kubernetes and assign a role to the user.


## Generate private key and CSR 

In [36]:
%env K8_USER=vikas-1


env: K8_USER=vikas-1


In [37]:
!openssl genrsa -out ${K8_USER}.key 2048

In [38]:
!openssl req -new -key ${K8_USER}.key -out ${K8_USER}.csr -subj "/CN=${K8_USER}/O=group1"

## Create a certificate signing request 

The CSR is geneated by the user and handed off to the admin - the admin takes care of signing the certificate and handing it back to the user.

In the example below the admin is creating a CertificateSigningRequest using the user's CSR.

In [40]:
import os 

base64_csr = os.popen(f"cat {os.environ['K8_USER']}.csr | base64 | tr -d '\n'").read()
csr_yaml = f"""---
apiVersion: certificates.k8s.io/v1
kind: CertificateSigningRequest
metadata:
  name: {os.environ['K8_USER']}
spec:
  groups:
  - system:authenticated
  request: {base64_csr}
  signerName: kubernetes.io/kube-apiserver-client
  usages:
  - client auth
"""

# write csr.yaml file
with open("/tmp/csr.yaml", "w") as f:
    f.write(csr_yaml)

!kubectl apply -f /tmp/csr.yaml

certificatesigningrequest.certificates.k8s.io/vikas-1 created


## Admin Will Approve the CSR 

Approve the CSR and generate a CRT file which will be used by the user to authenticate  


In [41]:
! kubectl certificate approve ${K8_USER}

certificatesigningrequest.certificates.k8s.io/vikas-1 approved


## Configure kubectl to use the new user and cert 

In [43]:
# Get certificate 
! kubectl get csr ${K8_USER} -o jsonpath='{.status.certificate}' | base64 --decode > ${K8_USER}.crt


# Create credentials for user in kubeconfig
! kubectl config set-credentials ${K8_USER} --client-certificate=${K8_USER}.crt --client-key=${K8_USER}.key


User "vikas-1" set.


Assign credentials to a cluster 

In [44]:
! kubectl config set-context ${K8_USER} --cluster=do-blr1-k8s-1-25-4-do-0-blr1-1677150099490 --user=${K8_USER}

Context "vikas-1" created.


In [47]:
# Using template assign_permissions.yaml.temp generate yaml file 
! sed "s/USER/${K8_USER}/g" templates/assign_permissions.templ > assign_permissions.yaml

! cat assign_permissions.yaml

# Bind cluster-admin role to user vikas-temp 
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: vikas-1-binding
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: cluster-admin
subjects:
- kind: User
  name: vikas-1
  apiGroup: rbac.authorization.k8s.io


In [ ]:
! kubectl apply -f assign_permissions.yaml

## Cleanup things 